In [1]:
import pandas as pd

## step 1: load OSM/GIS-exported names data

1(a) merge the files I have - right now, Africa, Central America, South America

In [70]:
csvs_to_merge = ["OSMNames_export_tables/Africa.csv", "OSMNames_export_tables/CentralAmerica.csv", "OSMNames_export_tables/SouthAmerica.csv"]
merged_df = pd.DataFrame()

for file in csvs_to_merge:
    df = pd.read_csv(file)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# can save it to a file too ... 
#merged_df.to_csv("merged_file.csv", index=False)
basins = merged_df["BCODE"].unique() 
basins.shape # there are 132 basins from these three regions

(132,)

1(a) NEW ONES - from May 3 ish, Asia, Europe, North America

In [45]:
as_eu_na = pd.read_csv("AS_EU_NA_frequency.csv")
basins_185 = as_eu_na['BCODE'].unique()
# len(basins_185) # there are 185, added to 132 gets us 317 - so there are 4 (?) in both sets.

In [46]:
# get the other sheet in here
af_sa_ca = pd.read_excel('AF_CA_SA_basin_terms.xlsx') # hard-coding the path is fine because I'm the only one running this, I think
basins_129 = af_sa_ca['BCODE'].unique() 
#check if there's this term in the list
#len(basins_129)

was curious about the overlap between the two datasets.

- should I merge them? yes :) 
- but! actually not until after some clean steps separately
- this is because asia has a bunch of non-roman characters

In [ ]:
# set1 = set(basins_185)
# set2 = set(basins_129)
# overlap_basins = set1.intersection(set2)
# print(overlap_basins)
# # curious if I should just merge in these overlap basins, remove them from eurasia/nam and add to af/cam
# overlap_df = as_eu_na[as_eu_na['BCODE'].isin(list(overlap_basins))]
#leftover_df = as_eu_na[~as_eu_na['BCODE'].isin(list(overlap_basins))]
# so these can tolerate a clean step - removing water-related and non-roman
#new_north = pd.DataFrame(leftover_df)

In [ ]:
#new_north.to_csv("new_north_may7.csv") # this is stored so that I can refer to it later without needing to run the above code
# and then uncomment and run later when I'm ready to deal with "new north"

#asia_europe_namerica = pd.read_csv("new_north_may7.csv")

## now what's left is new_south and new_north
- "south" refers to South America, Central America, and Africa
- "north" refers to North America, Europe, and Asia
- there were six basins {'SUCT', 'HOND', 'CTAT', 'GJLV', 'NILE', 'CDLR'} in both groups
- "new south" includes all overlap. those six basins are in new south.
- "new north" removes all overlap. those six basins are not in new north.
- 
- 
* -  this decision was made solely because (in early May) I have made progress on the "south" basins first (the files were smaller and easier to work with)
* -  another factor in this decision is that these overlap basins/names appeared to have similar languages (in the water-related terms) and similar presence of non-roman characters as the "south" basins, so I felt that our current work flow would apply
* -  new_north is likely to pose different challenges when it comes to non-roman characters.

In [73]:
new_south = pd.concat([merged_df, overlap_df], ignore_index=True)
new_south

,FREQUENCY,BCODE,name,waterway
0,6,AKPA,,drain
1,38,AKPA,,river
2,100,AKPA,,stream
3,3,AKPA,Afaẖ,stream
4,49,AKPA,Akpakorum,river
...,...,...,...,...
17532,4,SUCT,,ditch
17533,2,SUCT,,drain
17534,20,SUCT,,river
17535,50,SUCT,Río Suchiate,river


1(b) clean them up a little - drop nans, drop non roman alphabet results

In [102]:
#df_dropna = new_south.dropna(subset=['name']) # this actually doesn't work ...
#df_dropna = new_south[new_south.name != " "] # for some reason most of the nans are empty strings with a space
df_dropna = new_south[new_south.name != ""] # so these are empty strings with no space
df_dropna

,FREQUENCY,BCODE,name,waterway
3,3,AKPA,Afaẖ,stream
4,49,AKPA,Akpakorum,river
5,10,AKPA,Akpasang,river
6,19,AKPA,Ikpan,river
7,10,AKPA,Nkanya,river
...,...,...,...,...
17522,39,HOND,Hondo,river
17523,3,HOND,Ixnohá,river
17524,4,HOND,Seco,river
17526,4,HOND,Gold Button,stream


In [100]:
df_dropna

,FREQUENCY,BCODE,name,waterway
0,6,AKPA,,drain
1,38,AKPA,,river
2,100,AKPA,,stream
3,3,AKPA,Afaẖ,stream
4,49,AKPA,Akpakorum,river
...,...,...,...,...
17532,4,SUCT,,ditch
17533,2,SUCT,,drain
17534,20,SUCT,,river
17535,50,SUCT,Suchiate,river


In [50]:
# probably keep only terms in the roman alphabet since our terms list is too long
import re

'''
def keep_roman_chars(text): # this was removing single letters from words, changing the name entirely
    if isinstance(text, str):
        # this keeps only Latin/Roman characters, numbers, and common punctuation
        return re.sub(r'[^\x00-\x7F]+', '', text).strip()
    return text


#roman_africa_names = keep_roman_chars(Africa_names)
#roman_africa_names # don't quite know what to do with an array
'''

# this one gets a little more context before it drops the characters

def char_filter(text):
    if not isinstance(text, str):
        return text
        
    # Split text into words
    words = text.split()
    
    # Process each word
    processed_words = []
    for word in words:
        # Check if the word contains any ASCII characters
        if re.search(r'[\x00-\x7F]', word):
            # Word has some ASCII chars, keep it intact
            processed_words.append(word)
        else:
            # Word has only non-ASCII chars, skip it
            continue
    
    # Join the filtered words back together
    return ' '.join(processed_words).strip()


In [103]:
df_dropna["name"] = df_dropna["name"].apply(char_filter)

# I'm just getting rid of all the extra columns

#df_dropna[df_dropna.name != " "] # maybe don't need to do this

new_south = pd.DataFrame(df_dropna)


/var/folders/ns/5nr18wp5321282mzd8gc8cbm0000gn/T/ipykernel_3341/1477255765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna["name"] = df_dropna["name"].apply(char_filter)


In [104]:
new_south['name'].unique() # why are there still nans dude?????

array(['Afaẖ', 'Akpakorum', 'Akpasang', ..., 'Fogotico', 'La Florida',
       'Brenquio'], shape=(13642,), dtype=object)

## step 2: remove water-related terms
make a list of words that we'll remove, because they'll already be captured with our water related search terms

* (2a) get words from the extracted OSM terms above

* (2b) get words from our water-related search terms (Alabama meeting Feb 2025)

In [53]:
#import pandas as pd
febsearchterms = pd.read_excel("SearchTerms_Feb2025.xlsx")

box1_raw = febsearchterms.iloc[0, 1]

def parse_terms(raw_terms):
    # Split the string by OR and remove leading/trailing whitespace
    terms = [term.strip() for term in box1_raw.split('OR')]
    
    # Remove asterisks and any extra whitespace
    clean_terms = [re.sub(r'\*', '', term).strip() for term in terms]
    
    # Remove any empty strings and get unique terms
    unique_clean_terms = set(clean_terms)
    
    return unique_clean_terms

In [54]:
parsed_box1terms = parse_terms(box1_raw)
print(parsed_box1terms)

{'spring', 'creek', 'flood', 'riparian', 'tributar', 'pond', 'river', 'lake', 'drought', 'stream', 'hydroelect', 'dam', 'aquifer', 'reservoir', 'streams', 'water', 'wadi', 'oass', 'irrigat', 'canal', 'groundwater'}


* (2c) add in some translations for major languages

In [55]:
# went to google translate, asked for these terms above in the following common languages. removed duplicates.
# noting that I filled in *s (oas*s to oasis, for example) - this is where we could use help from a native speaker
box1_arabic = {'difafu', 'tyar', 'wahatu', 'khazan mayiy', "ma'", 'nhar', 'khur', 'jdawul', 'nbae', 'bhayratu', 'fiadani', 'miah jufiatun', 'kahrumayiyatun', 'khazan', 'ri', 'wadi', 'sad', 'barkata', 'jifaf', 'rafid', 'rwafdi', 'qnatu'}
box1_spanish = {'sequía', 'lago', 'inundación', 'arroyo', 'acuífero', 'ribereño', 'agua', 'manantial', 'hidroeléctrica', 'arroyo', 'arroyos', 'presa', 'irrigación', 'embalse', 'afluente', 'agua subterránea', 'río', 'estanque', 'uadi', 'quebrada', 'dren'}
box1_portuguese = {'seca', 'inundação', 'riacho', 'aquífero', 'ribeirão', 'água', 'nascente', 'hidrelétrica', 'riacho', 'riachos', 'barragem', 'irrigação', 'reservatório', 'água subterrânea', 'rio', 'lagoa', 'uádi', 'oásis'}
box1_french = {'sécheresse', 'lac', 'inondation', 'ruisseau', 'aquifère', 'rivière', 'eau', 'hydroélectricité', 'ruisseau', 'ruisseaux', 'barrage', 'irrigation', 'réservoir', 'affluent', 'eaux souterraines', 'rivière', 'étang', 'oued'}
box1_mandarin = {'Quánshuǐ','xiǎo xī','hóngshuǐ', "hé'àn", 'zhīliú', 'chítáng', 'héliú', 'húbó', 'gānhàn', 'xīliú', 'shuǐlì fādiàn','shuǐbà','hánshuǐ céng','shuǐkù','xīliú','shuǐ', 'gàn gǔ','lǜzhōu','guàngài','yùnhé','dìxiàshuǐ'}
box1_hindi = {"jharana", "khaadee", "baadh", "tatavartee", "sahaayak nadee", "taalaab", "nadee", "jheel", "sookha", "dhaara", "jalavidyut", "baandh", "jalabhrt", "jalaashay", "dhaaraen", "paanee", "vaadee", "nakhalistaan", "sinchaee", "nahar", "bhoojal"}
box1_russian = {'istochnik', 'ruchey', 'navodneniye', 'pribrezhnyy', 'pritok', 'prud', 'reka', 'ozero', 'zasukha', 'ruchey', 'gidroelektrostantsiya', 'plotina', 'vodonosnyy gorizont', 'vodokhranilishche', "ruch'i", 'voda', 'vadi', 'oazis', 'orosheniye', 'kanal', 'gruntovyye vody'}

# add arabic to parsed box 1
box1_translations = parsed_box1terms | box1_arabic | box1_spanish | box1_portuguese | box1_french | box1_mandarin | box1_hindi | box1_russian
# this | operator works like a + but with a set not a list
print(box1_translations)

{'spring', 'réservoir', 'inundação', 'húbó', 'sécheresse', 'yùnhé', 'shuǐ', 'sequía', 'riparian', 'ribereño', 'paanee', 'sinchaee', 'zasukha', 'shuǐkù', 'khazan mayiy', 'água', 'zhīliú', 'baadh', 'pritok', 'drought', 'stream', 'barragem', 'dhaara', 'inondation', 'irrigação', 'kanal', 'aquifer', 'difafu', 'baandh', 'jifaf', 'irrigación', 'rwafdi', 'nascente', 'agua', 'vodonosnyy gorizont', 'eaux souterraines', 'vodokhranilishche', 'ruisseaux', 'streams', 'nakhalistaan', 'voda', 'wadi', 'kahrumayiyatun', 'rio', 'jheel', 'gānhàn', 'navodneniye', 'hidrelétrica', 'oued', 'canal', 'jharana', 'khaadee', 'uádi', 'uadi', 'creek', 'reka', 'agua subterránea', 'água subterrânea', 'flood', 'tyar', 'xīliú', 'héliú', 'tributar', 'hóngshuǐ', 'qnatu', "ma'", 'khazan', 'manantial', 'vaadee', 'lake', 'orosheniye', 'sad', 'riachos', 'shuǐlì fādiàn', 'eau', "hé'àn", 'fiadani', 'miah jufiatun', 'rivière', 'dìxiàshuǐ', 'jdawul', 'bhayratu', 'irrigat', 'taalaab', 'prud', 'barkata', 'wahatu', 'sahaayak nadee',

## then step 3, going through the dataframe name column and removing any of the box 1 words

In [86]:
# this is older, and works on a list. the next cell has one that can work on single strings and be used through rows in dataframe

def clean_string(s, terms_to_remove):
    """
    Remove specified terms from within a string, case-insensitively,
    ensuring proper spacing after removal.
    
    Args:
        s (str): String to process
        terms_to_remove (list): Terms to remove
    
    Returns:
        str: Processed string with terms removed
    """
    if not isinstance(s, str):
        return s
    
    # Split into words
    words = s.split()
    
    # Filter out words that match any term in terms_to_remove (case-insensitive)
    cleaned_words = [
        word for word in words 
        if word.lower() not in [term.lower() for term in terms_to_remove]
    ]
    
    # Join back with a single space
    return ' '.join(cleaned_words).strip()

In [105]:
# Now apply to your DataFrame column using lambda with your list of terms to remove
terms_to_remove = box1_translations

# Apply to the DataFrame column
new_south['name'] = new_south['name'].apply(lambda x: clean_string(x, terms_to_remove))

In [108]:
new_south['BCODE'].unique()

array(['AKPA', 'ALHN', 'ANNL', 'AWSH', 'BENT', 'BIAX', 'BNSA', 'BRKA',
       'BUZI', 'CLNG', 'CNGO', 'CROS', 'CSTO', 'CVLY', 'DAUR', 'DRAX',
       'ETOS', 'GAMB', 'GASH', 'GEBA', 'GLNA', 'GSCR', 'GUIR', 'ICMT',
       'JUBA', 'KMOE', 'KUNE', 'LGPS', 'LKCH', 'LKCW', 'LKCY', 'LKDR',
       'LKNT', 'LKRW', 'LKTK', 'LMPO', 'LOFF', 'LSCR', 'MANA', 'MBEX',
       'MDJD', 'MOAX', 'MONO', 'MPUT', 'NGER', 'NILE', 'NYGA', 'ODBN',
       'OGOO', 'OKVG', 'ORAN', 'OUEM', 'PANG', 'PUNG', 'RVMA', 'SABI',
       'SANA', 'SASS', 'SENG', 'SJAF', 'SPAU', 'TAFN', 'TANO', 'UBLZ',
       'UMBA', 'UTBN', 'VOLT', 'ZAMB', 'ATBN', 'BLZE', 'CDLR', 'CGNL',
       'CHLT', 'COCO', 'CTAT', 'GJLV', 'GOSR', 'HOND', 'JURD', 'LKAZ',
       'LKEQ', 'LMPA', 'MASS', 'MOHO', 'MOTQ', 'NEGR', 'PAZX', 'PDNL',
       'SIOL', 'SJUA', 'SRTU', 'SUCT', 'TEMA', 'AMCR', 'AMZN', 'AYSN',
       'BAKR', 'BRMA', 'CHIC', 'CHIR', 'CHUY', 'CNCS', 'COMA', 'CRTY',
       'CTTB', 'CULL', 'ESQB', 'GALG', 'LKFN', 'LKTC', 'LMRM', 'LOAX',
      

In [109]:
new_south

,FREQUENCY,BCODE,name,waterway
3,3,AKPA,Afaẖ,stream
4,49,AKPA,Akpakorum,river
5,10,AKPA,Akpasang,river
6,19,AKPA,Ikpan,river
7,10,AKPA,Nkanya,river
...,...,...,...,...
17522,39,HOND,Hondo,river
17523,3,HOND,Ixnohá,river
17524,4,HOND,Seco,river
17526,4,HOND,Gold Button,stream


In [130]:
# and then I need to add this to the 
# old_south = pd.read_excel('AF_CA_SA_basin_terms.xlsx')
# updated_south = pd.concat([old_south, dropna_clean_df])

# # probably sort by bcode so they're all together, though maybe it doesn't matter
new_south = new_south.sort_values(by='BCODE')
new_south.to_excel('udpated_south_pre-edit.xlsx') # excel deals better with non-ascii than csv?


In [119]:
new_south['waterway'].unique()

array(['stream', 'river', '    river', '    stream', 'drain', 'canal',
       'dam', 'ditch', 'tidal_channel', '    ditch', '    canal',
       '    drain', 'rapids', 'fairway', '    weir', 'waterfall', 'weir',
       'dock', '    dam', 'wadi', '    waterfall', 'planned_canal'],
      dtype=object)

In [114]:
south_bcodes = new_south["BCODE"].unique()

In [125]:
new_south.iloc[1, 0]

np.int64(49)

In [126]:
type(new_south.iloc[1, 0])

numpy.int64

In [131]:
new_south[new_south['BCODE']=='SENO']

,FREQUENCY,BCODE,name,waterway
17155,7,SENO,de las Chinas,river
17159,1,SENO,Diente,river
17152,1,SENO,Ascencio,river
17153,11,SENO,Baguales,river
17151,4,SENO,Zanja Honda,river
17160,15,SENO,Don Guillermo,river
17157,1,SENO,del Arriero,river
17150,2,SENO,Ensenada de Falcón,river
17158,1,SENO,del Francés,river
17149,3,SENO,Cazador,stream


In [117]:
search_terms_official = pd.read_excel('/Users/selenawallace/Documents/geography/geography/search_terms.xlsx')
searchterms_south = search_terms_official[search_terms_official['BCODE'].isin(south_bcodes)]
searchterms_south['Basin_Name'].unique()

array(['Akpa', 'Amacuro', 'Amazon', 'Annole', 'Artibonite', 'Awash',
       'Aysen', 'Baker', 'Benito/Ntem', 'Bia', 'Belize',
       'Oued Bou Namoussa', 'Baraka', 'Barima', 'Buzi', 'Candelaria',
       'Changuinola', 'Carmen Silva/Chico', 'Chira', 'Choluteca', 'Chuy',
       'Chiloango', 'Cancoso/Lauca', 'Congo/Zaire', 'Coco/Segovia',
       'Comau', 'Cross', 'Corantijn/Courantyne', 'Cestos',
       'Coatan Achute', 'Catatumbo', 'Cullen', 'Cavally', 'Daoura', 'Dra',
       'Essequibo', 'Cuvelai/Etosha', 'Gallegos/Chico', 'Gambia', 'Gash',
       'Geba-Corubal', 'Grijalva', 'Galana', 'Goascoran',
       'Great Scarcies', 'Guir', 'Hondo', 'Incomati', 'Juba-Shibeli',
       'Jurado', 'Komoe', 'Kunene', 'Lotagipi Swamp', 'Lake Azuei',
       'Lake Chad', 'Lake Chilwa', 'Lake Cayo', 'Lak Dera',
       'Lake Enriquillo', 'Lake Fagnano', 'Lake Natron', 'Lake Rukwa',
       'Lake Titicaca-Poopo System', 'Lake Turkana', 'Lempa', 'Limpopo',
       'Lagoon Dos Patos-Lagoon Mirim', 'Loffa', 'La P

- step 3.5, do some manual cleaning before proceeding

## step 4, deal with duplicates, sort by frequency, bin by waterway type
-
steps to insert in the if/then statement checking search string length
* check for duplicates (last, after new string, sorting and binning)
* create new string BCODE-name-waterway
* sort by frequency (within bins)
* bin waterway types

In [227]:
# import cleaned from 3.5
south_cleaned = pd.read_excel("updated_south.xlsx")
# reset index
south_cleaned

,FREQUENCY,BCODE,name,waterway
0,3,AKPA,Afaẖ,stream
1,49,AKPA,Akpakorum,river
2,10,AKPA,Akpasang,river
3,19,AKPA,Ikpan,river
4,10,AKPA,Nkanya,river
...,...,...,...,...
16668,2,ZARM,Chiquita,river
16669,2,ZARM,Las Lajas,river
16670,6,ZARM,Palmales,stream
16671,2,ZARM,Faical,stream


- bin/categorize waterway types first

In [229]:

priority_map = {
        'river': 1,
        'stream': 2,
        'canal': 3,
        'dam': 4,
        'fairway' : 5
    }

# Create new column using map with a default value
south_cleaned['waterway_priority'] = south_cleaned['waterway'].map(priority_map).fillna(6).astype(int)
# and all not in map ("low priority" waterways) will be 6

In [230]:
# add a column that concatenates the strings in three columns (add - delimiter) BCODE-name-waterway
south_cleaned['combined'] = south_cleaned['BCODE'] + '-' + south_cleaned['name'] + '-' + south_cleaned['waterway_priority'].astype(str)
#south_cleaned


In [231]:
def deduplicate(df):
    # Step 1: Deduplicate by 'combined' and sum 'FREQUENCY'
    frequency_sums = df.groupby('combined')['FREQUENCY'].sum().reset_index()
    freq_dict = dict(zip(frequency_sums['combined'], frequency_sums['FREQUENCY']))
    
    # Select first row for each unique 'combined' value
    deduplicated_df = df.groupby('combined', as_index=False).first()
    
    # Update 'FREQUENCY' column with summed frequencies
    deduplicated_df['FREQUENCY'] = deduplicated_df['combined'].map(freq_dict)
    
    # Step 2: Calculate 'basin_freq_total' based on 'BCODE'
    basin_totals = deduplicated_df.groupby('BCODE')['FREQUENCY'].sum().reset_index()
    basin_dict = dict(zip(basin_totals['BCODE'], basin_totals['FREQUENCY']))
    
    # Add the 'basin_freq_total' column
    deduplicated_df['basin_freq_total'] = deduplicated_df['BCODE'].map(basin_dict)
    
    return deduplicated_df

deduplicated_df = deduplicate(south_cleaned)
#deduplicated_df

In [232]:
# then normalize
deduplicated_df['normalized_frequency'] = deduplicated_df['FREQUENCY']/deduplicated_df['basin_freq_total']
#deduplicated_df

In [233]:
sorted_df = deduplicated_df.sort_values(by=['BCODE', 'waterway_priority', 'normalized_frequency'], ascending=[True, True, False])
#sorted_df

## notes for north
* check what languages nexis uni will accept in search terms
* frequency priority and waterway weight as before
* second weight for nonroman vs roman characters (prioritize roman, cut nonroman)

* compare to TFDD previous search terms (which should be sorted priority)

In [234]:
# maybe just put it in a sheet normalized_south so it isn't gone forever lol
sorted_df.to_excel("normalized_south.xlsx")

## step 5, make the names into strings and check character limit
- each basin will have a unique cutoff point, cut off based on step 4 ordering/binning
- put it in a spreadsheet the script will be able to read

In [235]:
import re

def create_search_string(terms_list):
    # Since terms are already cleaned, just filter out empty/too short terms
    filtered_terms = []
    
    for term in terms_list:
        if isinstance(term, str) and term.strip() and len(term.strip()) > 1:
            filtered_terms.append(term.strip())
    
    # Remove duplicates while preserving order
    unique_terms = []
    for term in filtered_terms:
        if term not in unique_terms:
            unique_terms.append(term)
    
    # Format terms - add quotes only for multi-word terms
    formatted_terms = []
    for term in unique_terms:
        if ' ' in term:
            formatted_terms.append(f'"{term}"')  # Add quotes for multi-word terms
        else:
            formatted_terms.append(term)  # No quotes for single words
    
    # Join with OR
    search_string = " OR ".join(formatted_terms)
    
    return search_string

In [ ]:
# # testing for length
# febsearchterms = pd.read_excel("SearchTerms_Feb2025.xlsx") # this is in gitignore now

# box1 = febsearchterms.iloc[0, 1]
# box2 = febsearchterms.iloc[1, 1]
# #box3 = basin_specific_terms
# box4 = febsearchterms.iloc[4, 1] 

# string_wo_box3 = 'hlead(' + box1 + ') and hlead(' + box2 + ') and hlead(' + ') and not hlead(' + box4 + ')' # not including + box3

# box3_limit = 5000 - (len(string_wo_box3))

# box3_limit

3460

In [ ]:
# if I ever get rid of the above or move feb terms, just have it stored here, uncomment and run this
box3_limit = 3460

In [245]:
search_terms_data = []
#basins_to_trim = []

# a list of all the unique bcodes
basins_with_terms = sorted_df["BCODE"].unique()

for basin in basins_with_terms: # for each basin in that list
    df = sorted_df[sorted_df["BCODE"]== basin] #slice the dataframe so it's just the names in that basin
    list = df['name'].unique().tolist() # and make a list of 
    search_string = create_search_string(list) # this will make a search string out of it using the method above

    #then we check it for character limit
    if len(search_string)>box3_limit: # if it's too long

        truncated = search_string[:box3_limit] # trims it to exactly the limit
        last_or_pos = truncated.rfind(" OR ") # finds the nearest 'or'
        search_string = search_string[:last_or_pos] # truncates from there so the final term is a complete one

    else:
        pass
    
    search_terms_data.append({'basin': basin, 'basin_specific_terms': search_string})

In [238]:
search_terms_south = pd.DataFrame(search_terms_data)
search_terms_south.to_excel("south_searchterms_May12.xlsx")

from here below is old, but may be useful

In [ ]:
# sort terms by frequency & frequency null (or make null high '999') high to low
# bin frequency potentially , consider hierarchy of different waterway types
# and then figure out how much to trim

In [ ]:
# in these basins (each?), get count of unique frequency, count of unique waterway
# could get rid of ditch, stream with 1, etc.

In [ ]:
# another manual clean step, make sure TFDD name is in the list of names from OSM

0      ADIG
1      AKPA
2      ALKL
3      ALSK
4      AMCR
       ... 
309    ZARM
310    ALHN
311    EMSX
312    LOAX
313    GRND
Name: BCODE, Length: 314, dtype: object

## and put the new search terms into a new sheet

In [ ]:
# load the original search terms sheet (in repository)
searchterms_path = '/Users/selenawallace/Documents/geography/geography/basins_searchterms_tracking.xlsx'# hard-coding the path is fine because I'm the only one running this, I think
search_terms_official = pd.read_excel(searchterms_path)


search_terms_south # is the new 


# Iterate through your new dataframe and update the original
for index, row in search_terms_south.iterrows():
    bcode = row['basin']
    new_terms = row['basin_specific_terms']
    
    # Find matching rows in the original dataframe by BCODE
    mask = search_terms_official['BCODE'] == bcode
    
    # Update if there's a match
    if mask.any():
        # Update the Basin_Specific_Terms
        search_terms_official.loc[mask, 'Basin_Specific_Terms'] = new_terms
        
        # Mark as updated
        search_terms_official.loc[mask, 'Terms_updated_2025'] = 'Y'  

# Save the updated spreadsheet
search_terms_official.to_excel(searchterms_path, index=False)  # or .csv if preferred


## older

In [ ]:
# this is not necessary right now but was helpful 

# check for spaces in 'name' first

def check_spaces(df, column_name):
    """
    Checks for leading and trailing spaces in a specified column of a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to check.

    Returns:
        pd.DataFrame: A DataFrame containing rows with leading or trailing spaces in the specified column.
                          Returns an empty DataFrame if no spaces are found.
    """
    space_mask = df[column_name].str.contains(r'^\s+|\s+$', na=False)
    return df[space_mask]

# Example Usage

df = pd.DataFrame(south_cleaned)

column_to_check = 'name'
rows_with_spaces = check_spaces(df, column_to_check)

if not rows_with_spaces.empty:
    print(f"Rows with leading/trailing spaces in '{column_to_check}':")
    print(rows_with_spaces)
else:
    print(f"No leading/trailing spaces found in '{column_to_check}'.")